In [1]:
# 라이브러리 불러오기
import os
import json
import pandas as pd

# JSON 파일들이 있는 폴더 경로 설정
train_path = 'C:/Users/prohe/OneDrive/Desktop/github_manage/DCC/filtered_label/training_label'
validation_path = 'C:/Users/prohe/OneDrive/Desktop/github_manage/DCC/filtered_label/validation_label'

### (1) 유효한 label 폴더로부터 "응답자 ID"와 "스타일 선호 여부" 정보 추출

In [2]:
def preference(label_me, output_file):
    # JSON 파일들이 있는 폴더 경로 설정
    folder_path = label_me

    # 데이터프레임을 만들기 위한 리스트 초기화
    records = []

    # 폴더 내 모든 JSON 파일 처리
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.json'):
            file_path = os.path.join(folder_path, file_name)

            # JSON 파일 로드
            with open(file_path, 'r') as f:
                try:
                    data = json.load(f)

                    # 응답자 정보 (user 딕셔너리)
                    r_id = data['user']['R_id']

                    # item 딕셔너리에서 imgName과 Q5 값 추출
                    img_name = data['item']['imgName']
                    q5_value = data['item']['survey']['Q5']

                    # Q5 값에 따라 선호와 비선호 분류
                    if q5_value == 2:
                        preference = '스타일 선호'
                    elif q5_value == 1:
                        preference = '스타일 비선호'
                    records.append({'응답자 ID': r_id, '파일명': img_name, '스타일 선호 여부': preference})
                    

                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON from file {file_name}: {e}")
                except KeyError as e:
                    print(f"Missing key {e} in file {file_name}")

    # 데이터프레임 생성
    df = pd.DataFrame(records)

    # 응답자 ID 순서대로 정렬
    df.sort_values(by='응답자 ID', inplace=True)
    df.reset_index(drop=True, inplace=True)

    # 중복 데이터 확인
    duplicate_rows = df[df.duplicated()]
    # 중복이 존재하면 제거
    if not duplicate_rows.empty:
        df.drop_duplicates(inplace=True)
        print(f"중복 데이터 {len(duplicate_rows)}개가 제거되었습니다.")

    # 원하는 파일로 저장 (CSV 파일)
    df.to_csv(output_file, encoding='utf-8-sig', index=False)

    print(f"데이터가 {output_file}에 저장되었습니다.")
    
    return df  # 최종 데이터프레임 반환

In [3]:
# 함수 사용
df_train = preference(train_path, 'train_preference.csv')
df_val = preference(validation_path, 'val_preference.csv')

중복 데이터 12개가 제거되었습니다.
데이터가 train_preference.csv에 저장되었습니다.
중복 데이터 5개가 제거되었습니다.
데이터가 val_preference.csv에 저장되었습니다.


### (2) train/valid 둘 다 응답한 R_id를 기준으로 상위 100명의 데이터 선택

In [8]:
import pandas as pd

# csv 불러와서 사용
df_train = pd.read_csv('train_preference.csv')
df_val = pd.read_csv('val_preference.csv')

In [9]:
# 데이터 프레임의 형태 확인하기
df_train.head(10)

,응답자 ID,파일명,스타일 선호 여부
0,12,W_03412_50_classic_W.jpg,스타일 선호
1,25,W_12740_00_metrosexual_M.jpg,스타일 비선호
2,26,W_18990_50_feminine_W.jpg,스타일 비선호
3,27,W_06563_70_hippie_M.jpg,스타일 비선호
4,27,W_17260_19_normcore_M.jpg,스타일 선호
5,27,W_07347_90_hiphop_M.jpg,스타일 비선호
6,27,W_16068_80_bold_M.jpg,스타일 선호
7,27,W_15268_50_ivy_M.jpg,스타일 선호
8,27,W_17957_80_bold_M.jpg,스타일 선호
9,27,W_15212_60_mods_M.jpg,스타일 비선호


In [10]:
# 응답자 ID 별로 설문 응답의 개수 세기
train_count = df_train.groupby(['응답자 ID']).count()
val_count = df_val.groupby(['응답자 ID']).count()

In [11]:
# 하나의 열만 남기기
train_count = train_count['스타일 선호 여부']
val_count = val_count['스타일 선호 여부']

# 열 이름이 동일하므로 헷갈리지 않게 변경
train_count.name = 'train 설문 응답 수'
val_count.name = 'val 설문 응답 수'

# 두 데이터프레임 합친 후 몇 가지 처리
df_sum = pd.concat([train_count, val_count],axis=1)
df_sum = df_sum.fillna(0).astype(int)    # 결측치 0으로 채우기
df_sum['합계'] = df_sum['train 설문 응답 수'] + df_sum['val 설문 응답 수']    # '합계' 열 추가
df_sum = df_sum.sort_values(by='합계', ascending=False)    # '합계' 열 기준으로 내림차순 정렬

In [12]:
df_sum

,train 설문 응답 수,val 설문 응답 수,합계
응답자 ID,,,
64747,45,15,60
65139,46,12,58
64561,46,12,58
63405,44,14,58
64346,46,12,58
...,...,...,...
65084,1,0,1
65087,1,0,1
65088,1,0,1


In [13]:
#df_sum의 합계를 기준으로 상위 100개 응답자 ID 추출하여 리스트로 저장
top_100_ids = df_sum.head(100).index.tolist()

In [14]:
# 각 데이터에서 유효한 데이터만 거르기
top100_train_df = df_train[df_train['응답자 ID'].isin(top_100_ids)].reset_index(drop=True)
top100_val_df = df_val[df_val['응답자 ID'].isin(top_100_ids)].reset_index(drop=True)

In [15]:
# 각 데이터의 응답자 수가 100명인지 재확인
top100_train_df['응답자 ID'].nunique(), top100_val_df['응답자 ID'].nunique()

(100, 100)

In [16]:
top100_train_df

,응답자 ID,파일명,스타일 선호 여부
0,368,W_06753_60_mods_M.jpg,스타일 선호
1,368,W_06686_70_hippie_M.jpg,스타일 선호
2,368,W_15453_70_hippie_M.jpg,스타일 비선호
3,368,W_06843_60_mods_M.jpg,스타일 선호
4,368,W_06896_10_sportivecasual_M.jpg,스타일 선호
...,...,...,...
4445,67975,T_17798_19_normcore_M.jpg,스타일 선호
4446,67975,T_17799_19_normcore_M.jpg,스타일 선호
4447,67975,W_06915_00_metrosexual_M.jpg,스타일 비선호
4448,67975,T_17801_19_normcore_M.jpg,스타일 선호


In [17]:
# pivot table을 합치기 위한 준비 단계

# dataset에 대한 state 정보를 추가
top100_train_df['dataset'] = 'Training'
top100_val_df['dataset'] = 'Validation'

# 두 데이터프레임 합치기
combined_df = pd.concat([top100_train_df, top100_val_df], ignore_index=True)

# '응답자 ID'와 'dataset'을 기준으로 그룹화하고, 파일명을 개행문자("\n")와 함께 합치기
grouped = combined_df.groupby(['응답자 ID', 'dataset', '스타일 선호 여부']).agg({
    '파일명': lambda x: '\n'.join(x)
}).reset_index()

# '스타일 선호 여부의 순서를 바꾸어서 'like'가 먼저 나오도록 정렬
grouped.sort_values(by=['응답자 ID', 'dataset', '스타일 선호 여부'], ascending=[True, True, False], inplace=True)

In [18]:
grouped

,응답자 ID,dataset,스타일 선호 여부,파일명
1,368,Training,스타일 선호,W_06753_60_mods_M.jpg\nW_06686_70_hippie_M.jpg...
0,368,Training,스타일 비선호,W_15453_70_hippie_M.jpg\nW_12459_50_ivy_M.jpg\...
3,368,Validation,스타일 선호,W_04678_50_ivy_M.jpg\nW_06864_10_sportivecasua...
2,368,Validation,스타일 비선호,W_16848_19_normcore_M.jpg\nW_15340_50_ivy_M.jp...
5,837,Training,스타일 선호,W_00829_10_sportivecasual_M.jpg\nW_00028_50_iv...
...,...,...,...,...
390,66731,Validation,스타일 비선호,W_29733_10_sportivecasual_W.jpg\nW_33901_90_ki...
392,67975,Training,스타일 선호,W_71936_60_mods_M.jpg\nT_21987_70_hippie_M.jpg...
391,67975,Training,스타일 비선호,W_26965_90_hiphop_M.jpg\nW_52586_50_ivy_M.jpg\...
394,67975,Validation,스타일 선호,W_17738_80_bold_M.jpg\nW_52578_50_ivy_M.jpg\nT...


In [19]:
# 피벗 테이블 생성
final_result = grouped.pivot(index='응답자 ID', columns=['dataset', '스타일 선호 여부'], values='파일명')

In [20]:
final_result

dataset                                             Training  \
스타일 선호 여부                                             스타일 선호   
응답자 ID                                                         
368        W_06753_60_mods_M.jpg\nW_06686_70_hippie_M.jpg...   
837        W_00829_10_sportivecasual_M.jpg\nW_00028_50_iv...   
7658       W_08708_10_sportivecasual_W.jpg\nW_06118_10_at...   
7905       W_24661_60_mods_M.jpg\nW_24765_60_mods_M.jpg\n...   
9096       W_03784_00_cityglam_W.jpg\nW_03984_90_hiphop_W...   
...                                                      ...   
66469      W_58816_00_metrosexual_M.jpg\nW_56363_10_sport...   
66513      W_64791_90_hiphop_W.jpg\nW_44554_90_grunge_W.j...   
66592      T_07897_00_cityglam_W.jpg\nT_05759_19_lounge_W...   
66731      T_00770_60_minimal_W.jpg\nT_10713_19_normcore_...   
67975      W_71936_60_mods_M.jpg\nT_21987_70_hippie_M.jpg...   

dataset                                                       \
스타일 선호 여부                                            스타일 비선호   
응답자 ID                                                         
368        W_15453_70_hippie_M.jpg\nW_12459_50_ivy_M.jpg\...   
837        W_12695_50_ivy_M.jpg\nW_24413_70_hippie_M.jpg\...   
7658       W_11169_19_normcore_W.jpg\nW_14023_90_lingerie...   
7905       W_25255_70_hippie_M.jpg\nW_33184_00_metrosexua...   
9096       W_10686_50_classic_W.jpg\nW_00191_10_sportivec...   
...                                                      ...   
66469      T_06076_60_mods_M.jpg\nT_07605_00_metrosexual_...   
66513      W_43040_70_punk_W.jpg\nW_38863_60_minimal_W.jp...   
66592      W_20883_70_disco_W.jpg\nT_07452_50_classic_W.j...   
66731      T_11540_90_lingerie_W.jpg\nT_14071_19_lounge_W...   
67975      W_26965_90_hiphop_M.jpg\nW_52586_50_ivy_M.jpg\...   

dataset                                           Validation  \
스타일 선호 여부                                             스타일 선호   
응답자 ID                                                         
368        W_04678_50_ivy_M.jpg\nW_06864_10_sportivecasua...   
837        W_17305_70_hippie_M.jpg\nW_00829_10_sportiveca...   
7658       W_09731_19_genderless_W.jpg\nW_04927_50_femini...   
7905            W_32034_80_bold_M.jpg\nW_02845_60_mods_M.jpg   
9096       W_18714_90_kitsch_W.jpg\nW_19075_50_classic_W....   
...                                                      ...   
66469      T_00456_10_sportivecasual_M.jpg\nT_01123_90_hi...   
66513                               W_14828_50_classic_W.jpg   
66592      T_00253_60_popart_W.jpg\nW_46907_80_powersuit_...   
66731                                                    NaN   
67975      W_17738_80_bold_M.jpg\nW_52578_50_ivy_M.jpg\nT...   

dataset                                                       
스타일 선호 여부                                            스타일 비선호  
응답자 ID                                                        
368        W_16848_19_normcore_M.jpg\nW_15340_50_ivy_M.jp...  
837        W_15661_70_hippie_M.jpg\nW_27700_70_hippie_M.j...  
7658       W_19003_50_feminine_W.jpg\nW_10510_60_space_W....  
7905       W_26179_60_mods_M.jpg\nW_28964_19_normcore_M.j...  
9096       W_14225_50_feminine_W.jpg\nW_05941_60_popart_W...  
...                                                      ...  
66469      W_58887_00_metrosexual_M.jpg\nW_24553_70_hippi...  
66513      W_10984_50_feminine_W.jpg\nW_37404_60_space_W....  
66592      W_34436_60_space_W.jpg\nW_02170_50_feminine_W....  
66731      W_29733_10_sportivecasual_W.jpg\nW_33901_90_ki...  
67975      W_17747_80_bold_M.jpg\nW_60184_10_sportivecasu...  

[100 rows x 4 columns]

In [21]:
# HTML로 보기 좋게 출력
from IPython.display import HTML

def display_df(df, rows=5):
    styles = [
        dict(selector="th", props=[("text-align", "left")]),
        dict(selector="td", props=[("white-space", "pre-wrap")])
    ]
    return HTML(df.head(rows).style.set_table_styles(styles).to_html())

# 100명의 응답자에 대한 결과를 모두 출력하면 너무 길어지므로 상위 10명만 출력
display(display_df(final_result, rows=10))